In [3]:
pip install --upgrade spark_ai_python

Looking in indexes: http://mirrors.cloud.aliyuncs.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
from sparkai.llm.llm import ChatSparkLLM, ChunkPrintHandler
from sparkai.core.messages import ChatMessage

In [9]:
pip install dotenv

Looking in indexes: http://mirrors.cloud.aliyuncs.com/pypi/simple/
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [69 lines of output]
      /root/miniconda3/lib/python3.12/site-packages/setuptools/dist.py:452: SetuptoolsDeprecationWarning: Invalid dash-separated options
      !!
      
              ********************************************************************************
              Usage of dash-separated 'index-url' will not be supported in future
              versions. Please use the underscore name 'index_url' instead.
      
              This deprecation is overdue, please update your project and remove deprecated
              calls to avoid build errors in the future.
      
              See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
              ******************************************************************

In [71]:
import  os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [19]:
print(os.environ["SPARK_API_SECRET"])

OThjNzVjZjJmZTQ1ZjY1ZGI1ODE3MzF


In [37]:
!pip install django.utils.formats

Looking in indexes: http://mirrors.cloud.aliyuncs.com/pypi/simple/
ERROR: Could not find a version that satisfies the requirement django.utils.formats (from versions: none)
ERROR: No matching distribution found for django.utils.formats


In [49]:
from datetime import datetime
from time import mktime
from wsgiref.handlers import format_date_time
import zoneinfo
import hmac
import hashlib
import base64
from urllib.parse import urlencode
from urllib.parse import urlparse

# 获取当前时间
cur_time = datetime.now()

# 设置时区为上海（CST，UTC+8）
shanghai_tz = zoneinfo.ZoneInfo("Asia/Shanghai")

# 将当前时间转换为上海时区
localized_time = cur_time.replace(tzinfo=shanghai_tz)

# 转换为时间戳
timestamp = mktime(localized_time.timetuple())

# 格式化时间戳
date = format_date_time(timestamp)

tmp = "host: " + "spark-api.xf-yun.com" + "\n"
tmp += "date: " + date + "\n"
tmp += "GET " + "/v4.0/chat" + " HTTP/1.1"
tmp_sha = hmac.new(os.environ["SPARK_API_SECRET"].encode('utf-8'), tmp.encode('utf-8'), digestmod=hashlib.sha256).digest()
signature = base64.b64encode(tmp_sha).decode(encoding='utf-8')
authorization_origin = f"api_key='{os.environ["SPARK_API_KEY"]}', algorithm='hmac-sha256', headers='host date request-line', signature='{signature}'"
authorization = base64.b64encode(authorization_origin.encode('utf-8')).decode(encoding='utf-8')
v = {
		"authorization": authorization, # 上方鉴权生成的authorization
        "date": date,  # 步骤1生成的date
    	"host": "wss://spark-api.xf-yun.com" # 请求的主机名，根据具体接口替换
}
url = "wss://spark-api.xf-yun.com/{}/chat?" + urlencode(v)

In [51]:
print(urlparse('wss://spark-api.xf-yun.com/v4.0/chat').path)

/v4.0/chat


In [15]:
# 定义spark Spark4.0 Ultra SDK
def gen_spark_params(model):
    '''
    构造星火大模型参数
    '''
    model_params_dict = {
        # v1.5 版本
        "v1.5": {
            "domain": "general", # 用于配置大模型版本
            "spark_url": url.format("v1.1") # 云端环境的服务地址
        },
        # v2.0 版本
        "v2.0": {
            "domain": "generalv2", # 用于配置大模型版本
            "spark_url": url.format("v2.1") # 云端环境的服务地址
        },
        # v3.0 版本
        "v3.0": {
            "domain": "generalv3", # 用于配置大模型版本
            "spark_url": url.format("v3.1") # 云端环境的服务地址
        },
        # v3.5 版本
        "v3.5": {
            "domain": "generalv3.5", # 用于配置大模型版本
            "spark_url": url.format("v3.5") # 云端环境的服务地址
        },
        # v4.0 版本
        "v4.0": {
            "domain": "generalv4.0", # 用于配置大模型版本
            "spark_url": url.format("v4.0") # 云端环境的服务地址
        }
    }
    return model_params_dict[model]

In [16]:
def get_completion(prompt, model='v4.0', temperature=0.1):
   '''
   获取星火大模型的调用结果

   请求参数：
        prompt：对应提示词
        model：调用的模型，默认为v4.0，也可以根据实际需求选择合适的模型
        temperature：模型输出的温湿度系数，
   ''' 
   spark = ChatSparkLLM(
        spark_api_url=gen_spark_params(model)["spark_url"],
        spark_app_id=os.environ["SPARK_APPID"],
        spark_api_key=os.environ["SPARK_API_KEY"],
        spark_api_secret=os.environ["SPARK_API_SECRET"],
        spark_llm_domain=gen_spark_params(model)["domain"],
        streaming=False,
    )
   messages = [ChatMessage(
        role="user",
        content=prompt
    )]
   handler = ChunkPrintHandler()
   response = spark.generate([messages], callbacks=[handler])
   return response

In [62]:
get_completion("你好")

SparkAIConnectionError: Handshake status 401 Unauthorized -+-+- {'date': 'Thu, 12 Dec 2024 18:14:19 GMT', 'content-type': 'application/json; charset=utf-8', 'connection': 'keep-alive', 'content-length': '43', 'server': 'kong/1.3.0'} -+-+- b'{"message":"HMAC signature does not match"}'

In [59]:
import _thread as thread
import os
import time
import base64

import base64
import datetime
import hashlib
import hmac
import json
from urllib.parse import urlparse
import ssl
from datetime import datetime
from time import mktime
from urllib.parse import urlencode
from wsgiref.handlers import format_date_time

import websocket
from concurrent.futures import ThreadPoolExecutor, as_completed

class Ws_Param(object):
    # 初始化
    def __init__(self, APPID, APIKey, APISecret, gpt_url):
        self.APPID = APPID
        self.APIKey = APIKey
        self.APISecret = APISecret
        self.host = urlparse(gpt_url).netloc
        self.path = urlparse(gpt_url).path
        self.gpt_url = gpt_url

    # 生成url
    def create_url(self):
        # 生成RFC1123格式的时间戳
        now = datetime.now()
        date = format_date_time(mktime(now.timetuple()))

        # 拼接字符串
        signature_origin = "host: " + self.host + "\n"
        signature_origin += "date: " + date + "\n"
        signature_origin += "GET " + self.path + " HTTP/1.1"

        # 进行hmac-sha256进行加密
        signature_sha = hmac.new(self.APISecret.encode('utf-8'), signature_origin.encode('utf-8'),
                                 digestmod=hashlib.sha256).digest()

        signature_sha_base64 = base64.b64encode(signature_sha).decode(encoding='utf-8')

        authorization_origin = f'api_key="{self.APIKey}", algorithm="hmac-sha256", headers="host date request-line", signature="{signature_sha_base64}"'

        authorization = base64.b64encode(authorization_origin.encode('utf-8')).decode(encoding='utf-8')

        # 将请求的鉴权参数组合为字典
        v = {
            "authorization": authorization,
            "date": date,
            "host": self.host
        }
        # 拼接鉴权参数，生成url
        url = self.gpt_url + '?' + urlencode(v)
        # 此处打印出建立连接时候的url,参考本demo的时候可取消上方打印的注释，比对相同参数时生成的url与自己代码生成的url是否一致
        return url


# 收到websocket错误的处理
def on_error(ws, error):
    print("### error:", error)


# 收到websocket关闭的处理
def on_close(ws):
    print("### closed ###")


# 收到websocket连接建立的处理
def on_open(ws):
    thread.start_new_thread(run, (ws,))


def run(ws, *args):
    data = json.dumps(gen_params(appid=ws.appid, query=ws.query, domain=ws.domain))
    ws.send(data)


# 收到websocket消息的处理
def on_message(ws, message):
    # print(message)
    data = json.loads(message)
    code = data['header']['code']
    if code != 0:
        print(f'请求错误: {code}, {data}')
        ws.close()
    else:
        choices = data["payload"]["choices"]
        status = choices["status"]
        content = choices["text"][0]["content"]
        print(content,end='')
        if status == 2:
            print("#### 关闭会话")
            ws.close()


def gen_params(appid, query, domain):
    """
    通过appid和用户的提问来生成请参数
    """

    data = {
        "header": {
            "app_id": appid,
            "uid": "1234",           
            # "patch_id": []    #接入微调模型，对应服务发布后的resourceid          
        },
        "parameter": {
            "chat": {
                "domain": domain,
                "temperature": 0.5,
                "max_tokens": 4096,
                "auditing": "default",
            }
        },
        "payload": {
            "message": {
                "text": [{"role": "user", "content": query}]
            }
        }
    }
    return data

def main(appid, api_secret, api_key, Spark_url, domain, query):
    wsParam = Ws_Param(appid, api_key, api_secret, Spark_url)
    websocket.enableTrace(False)
    wsUrl = wsParam.create_url()

    ws = websocket.WebSocketApp(wsUrl, on_message=on_message, on_error=on_error, on_close=on_close, on_open=on_open)
    ws.appid = appid
    ws.query = query
    ws.domain = domain
    ws.run_forever(sslopt={"cert_reqs": ssl.CERT_NONE})

In [ ]:
main(os.environ["SPARK_APPID"], os.environ["SPARK_API_SECRET"],os.environ["SPARK_API_KEY"],"wss://spark-api.xf-yun.com/v4.0/chat")


In [69]:
main(
        appid=os.environ["SPARK_APPID"],
        api_secret='OThjNzVjZjJmZTQ1ZjY1ZGI1ODE3MzFk',
        api_key=os.environ["SPARK_API_KEY"],
        #appid、api_secret、api_key三个服务认证信息请前往开放平台控制台查看（https://console.xfyun.cn/services/bm35）
        #Spark_url="wss://spark-api.xf-yun.com/v4.0/chat",      # Max环境的地址   
		Spark_url = "wss://spark-api.xf-yun.com/v4.0/chat", # 4.0Ultra环境的地址
        # Spark_url = "wss://spark-api.xf-yun.com/v3.1/chat"  # Pro环境的地址
        # Spark_url = "wss://spark-api.xf-yun.com/v1.1/chat"  # Lite环境的地址
        domain="4.0Ultra",     # Max版本
		# domain = "4.0Ultra"     # 4.0Ultra 版本
        # domain = "generalv3"    # Pro版本
        # domain = "lite"      # Lite版本址
        query="给我写一篇100字的作文"
    )

您好！您没有提供具体的作文主题，所以我将随机选择一个常见的话题来撰写这篇100字的作文。
标题：春日漫步

春风轻拂，万物复苏，我踏上了春日的漫步之旅。阳光透过嫩绿的叶缝，洒在蜿蜒的小径上，每一步都踏着温暖的光斑。花香袭人，蝴蝶翩翩起舞，小鸟在枝头欢唱，一切都显得那么生机勃勃。我深吸一口清新的空气，感受着春天的气息，心中的烦恼似乎也随之消散。这一刻，我与大自然融为一体，心灵得到了前所未有的宁静和满足。

希望您喜欢这篇简短的作文！如果您有其他特定的主题或要求，请随时告诉我。#### 关闭会话
### error: on_close() takes 1 positional argument but 3 were given


In [74]:
print(os.environ["SPARK_API_SECRET"])

OThjNzVjZjJmZTQ1ZjY1ZGI1ODE3MzF
